# Introdução (não-extensiva) a Online Learning


**Saulo Martiello Mastelini** (mastelini@usp.br)

Outras redes:

- [Github](https://github.com/smastelini)
- [Linkedin](https://www.linkedin.com/in/smastelini/) (eu deveria, mas não atualizo frequentemente -- para ser sincero, está bem desatualizado)
- [ResearchGate](https://www.researchgate.net/profile/Saulo-Mastelini)

MBA em Ciência de Dados<br>
Universidade de São Paulo, São Carlos, Brasil<br>
Copyright (c) 2021

---

**Disclaimer**

Como o título já diz, essa não é, de forma alguma, uma introdução extensiva ao tema. É apenas a minha humilde tentativa de dar um panorama geral de décadas de pesquisa em uma área que está em constante expansão e inovação.

Feliz ou infelizmente, **todo processo de aprendizado leva tempo**. Assim, não será possível aprofundar em toda uma área de pesquisa de décadas em apenas algumas horas.

Pretendo me inspirar na busca em estruturas de árvores (teoria dos grafos) para guiar nosso estudo. Vou mostrar três opções (a última customizada ao meu gosto), das quais as duas primeiras eu quero evitar:

1. **Busca em largura:** corremos o risco de não abordar tudo o que há de mais popular e, ainda por cima, sermos tão superficiais que os conteúdos seriam de pouca valia.
2. **Busca em profundidade:** aqui o risco é querer focar muito em um minúcias ou em um ponto específico e acabar não tendo um panorama geral das coisas.
3. **Busca em profundidade limitada** (e adaptativa) **:** Nos aprofundaremos um pouco em cada tópico, não muito, mas se quiserem, podemos nos aprofundar mais. Cabe a vocês pedirem! No entanto, me reservo ao direito de arbitrar o quanto devemos insistir num assunto durante nosso escasso tempo: buscarei balancear interesses individuais e o coletivo, bem como julgar o que pode ou não ajudar no estudo individual de cada um.

---

**Ponto de partida para aprofundamento**

Para quem quiser se aprofundar mais nos assuntos que eu vou apresentar hoje, eu sugiro os seguintes recursos:

- [Livro do MOA](https://moa.cms.waikato.ac.nz/book-html/): um livro de acesso aberto que abrange vários tópicos ligados a stream learning
- [Documentação do River](https://riverml.xyz/dev/): conta com vários exemplos e tutoriais, além de referenciais práticos e teóricos! Está sempre sendo expandida, corrigida e atualizada.

Quem tiver alguma dúvida específica que a documentação não possa suprir pode sempre abrir uma "*Discussion*" no Github. É só ir no [repositório do Github do River](https://github.com/online-ml/river) e clicar na aba de discussões. Alguém da equipe de desenvolvimento ou da comunidade de usuários do River com grande certeza estará disposto a ajudar!

Contribuições são bem vindas! O River é um projeto *Open Source* construído pela comunidade. Mesmo que alguém possa não ter conhecimento técnico dos algoritmos, sempre é possível ajudar! Correções e expansões na documentação, por exemplo, são formas de ajudar! Muitas vezes apontar erros ou bugs (através de *issues*) já é de grande ajuda! 😁

---

**Sobre o River**

O River é um projeto open source para online learning e stream mining. Ele surgiu a partir da fusão de duas ferramentas (também open-source):

- creme
- scikit-multiflow

Ambos os pacotes Python abordavam o mesmo tema, mas com enfoques específicos. Após um longo tempo de planejamento e conversas, os mantenedores e times de desenvolvimento dos dois pacotes uniram forças e lançaram o River. O novo pacote é o produto dos anos de lições aprendidas no desenvolvimento das ferramentas predecessoras.

O River reune o que havia de melhor `creme` e `scikit-multiflow` (que foram descontinuados) e supera pontos fracos que existiam nos projetos anteriores. A nova ferramenta é voltada tanto para o uso no mercado quanto na academia.

O River conta com muitos contribuidores, mas a maioria do time de desenvolvimento está distribuído entre a França, Nova Zelândia e Brasil.

O artigo da ferramenta no JMLR pode ser acessado [aqui](https://www.jmlr.org/papers/v22/20-1380.html).

O site do River é o: https://riverml.xyz

Existe muita demanda e nem tanto tempo 😅. Se quiserem checar os planos para o futuro, visitem o nosso [Roadmap](https://www.notion.so/d1e86fcdf21e4deda16eedab2b3361fb?v=503f44740b8b44a99a961aa96e9e46e1).

---

## Sumário
1. Online learning? Why?
2. Batch vs. Online
3. Estruturas necessárias: exemplo
    - Mean
    - Var
    - Quantile
4. Por que usar dicionários?
    - Exemplos com dados faltantes
5. Como avaliar um modelo?
    - `progressive_val_score`
    - label delay
6. Concept drift
7. Exemplos de algoritmos
    1. Classificação
        1. Logistic regression
        2. Hoeffding Tree
        3. Naive Bayes
        4. Adaptive Random Forest
        5. Streaming Random Patches
    2. Regressão
        1. Linear Regression
        2. **Hoeffding Tree**
        3. **AMRules**
        4. **Adaptive Random Forest**
        5. **Streaming Random Patches**
    3. Clustering
        1. k-Means
    4. Anomaly detection
        1. Half-space Trees
8. Expert module
9. Pipelines e pré-processamento
    - Encoding
    - Scaling
    - Filtragem e Seleção
    - Aritmética com Pipelines
    - Visualizando as coisas
10. Exemplo completo
    - processamento em tempo real
    - inspeção de modelos

In [ ]:
# Esses são os pacotes que estão sendo utilizados nesse notebook
# As linhas a seguir podem ser descomentadas para instalar o que for necessário

# !pip install numpy
# !pip install scikit-learn

# Ultima versão lançada
# !pip install river

# Eu estou usando a versão de desenvolvimento, que pode ser instalada com:
#!pip install git+https://github.com/online-ml/river --upgrade

# 1. Online Learning? Why?

Por que alguém deveria se preocupar em atualizar modelos a todo tempo? Não é só treinar e sair usando?

R: sim! Em grande parte dos casos isso é verdade.

Mas imagine que:

- A quantidade de dados produzidas é enorme
- Não é possível armazenar tudo
- Existe limitação no poder computacional para treinamento dos modelos
    - processador
    - memória
    - bateria
- Os dados são não-estacionários e/ou evoluem com o tempo

Nesses casos, posso usar aprendizado de máquina tradicional? R: sim!!

É possível usar aprendizado de máquina tradicional se:
- Os dados são estacionários (uma amostra representativa dos dados é suficiente)

ou

- A velocidade de produção dos dados não é tão alta (batch-incremental e os recursos computacionais são suficientes)

## 1.1 Batch-incremental


Um modelo tradicional de aprendizado de máquina é re-treinado de tempos em tempos. Para tal, uma janela para treinamento precisa ser definida.

Tipos de janelamento dos dados:


<img src="time_windows.png">

**Fonte:** Adaptado de:

> Carnein, M. and Trautmann, H., 2019. Optimizing data stream representation: An extensive survey on stream clustering algorithms. Business & Information Systems Engineering, 61(3), pp.277-297.

- *Landmarks* são a escolha mais comum em estratégias batch-incremental. É preciso definir o tamanho da janela de forma adequada.
    - O modelo pode ficar defasado se a janela for muito grande 
    - Ou o modelo pode não capturar os padrões se a janela for muito pequena
    - Concept-drift é um problema e tanto
    
**Atenção**: batch-incremental != mini-batch.
Redes neurais podem ser treinadas de forma incremental ou progressiva. No entanto, questões como "esquecimento catastrófico" são problemáticas. Esse e outros tipos de desafios são tratados na área de pesquisa chamada **continual learning**.

## 1.2 Importante lembrar

Data streams não são, necessariamente, séries temporais! 🤯

Mas qual é a diferença, então?

Basicamente, em streams não necessariamente temos dependências temporais explícitas como em séries temporais. Por exemplo: rede de sensores.

Os dados chegam temporalmente organizados, mas cada sensor tem uma taxa de transmissão específica, ou um delay específico. Alguns sensores pode falhar... outros serem adicionados. E assim por diante.

A ordem de chegada não importa... tanto, mas importa.

# 2. Batch vs. Online

Uma boa introdução acerca da "migração" de batch para online está disponível no [site](https://riverml.xyz/latest/examples/batch-to-online/) do River. Aqui eu só quero dar uma visão bem geral das diferenças. Entraremos em mais detalhes posteriormente nas minúcias de avaliação de modelos em Online Learning.


Uma possível validação de um modelo tradicional de aprendizado de máquina poderia ter essa "cara":

In [ ]:
from sklearn.datasets import load_wine
from sklearn.metrics import accuracy_score
from sklearn.model_selection import KFold
from sklearn.tree import DecisionTreeClassifier

data = load_wine()

X, y = data.data, data.target
kf = KFold(shuffle=True, random_state=8, n_splits=10)

accs = []

for train, test in kf.split(X):
    X_tr, X_ts = X[train], X[test]
    y_tr, y_ts = y[train], y[test]
    
    dt  = DecisionTreeClassifier(max_depth=5, random_state=93)
    dt.fit(X_tr, y_tr)
    
    accs.append(accuracy_score(y_ts, dt.predict(X_ts)))

print(f"Acurácia média: {sum(accs) / len(accs)}")

Reparem que todo o conjunto de dados está disponível e carregado em memória. O algoritmo de árvore de decisão pode percorrer os dados de treino quantas vezes forem necessárias. Os dados de teste (validação, no nosso caso) nunca são utilizados para treino.

No fim das contas, usaríamos o conjunto completo para treinar um modelo final (supondo que já encontramos um conjunto adequado de hiper-parâmetros). Uma vez treinado, esse modelo seria utilizado para fazer predições para novas amostras (de vinhos, nesse caso).

In [ ]:
from river import metrics
from river import stream
from river import tree


acc = metrics.Accuracy()
ht = tree.HoeffdingTreeClassifier(max_depth=5, grace_period=20)

for x, y in stream.iter_sklearn_dataset(load_wine()):
    # Métrica atualizada antes do treinamento
    acc.update(y, ht.predict_one(x))
    # Modelo treinado amostra-a-amostra
    ht.learn_one(x, y)

print(f"Acurácia: {acc.get()}")

Aqui, estamos percorrendo cada exemplo do conjunto de dados de forma sequencial. Os exemplos poderiam ser carregados diretamente do disco, de algum webservice, ou de onde sua imaginação te levar, sem a necessidade de salvar em memória nada.

Cada amostra é primeiramente utilizada para teste e depois passada para o modelo. Tudo é atualizado uma amostra por vez. Notem que não podemos comparar diretamente os desempenhos obtidos porque a estratégia de avaliação foi diferente nos dois casos.

Poderíamos ainda (pseudo) embaralhar os dados antes de passar para o modelo, se temos plena confiança que os dados são estacionários.

# 3. Estruturas necessárias: exemplo

In [ ]:
import numpy as np
from river import stats

Dado esse choque inicial, vamos prosseguir com calma e primeiramente pensar em alguns aspectos importantes antes de vermos os algoritmos de aprendizado de máquina online.

## 3.1. Um exemplo, para aquecimento cerebral

Vamos supor que queremos calcular estatísticas para dados que chegam a todo momento:

- Média
- Variância
- ...

Hora de simular:

In [ ]:
import random

rng = random.Random(42)

In [ ]:
%%time

valores = []
stds_batch = []

for _ in range(50000):
    v = rng.gauss(5, 3)
    valores.append(v)
    
    stds_batch.append(np.std(valores))

Por padrão o numpy calcula o desvio padrão populacional! Para usar a versão amostral precisamos passar `ddof=1` como parâmetro.

In [ ]:
rng = random.Random(42)

In [ ]:
%%time

stds_incr = []
var = stats.Var(ddof=0)

for _ in range(50000):
    v = rng.gauss(5, 3)
    var.update(v)
    stds_incr.append(var.get() ** 0.5)

Será que dá alguma diferença? E será que funciona?

In [ ]:
s_erros = 0

for batch, incr in zip(stds_batch, stds_incr):
    s_erros += (batch - incr)

s_erros, s_erros / len(stds_batch)

Ok... parece convincente. Mas e se o cenário fosse diferente? Se ao invés de calcularmos o desvio padrão e irmos aumentando a quantidade de dados, quisessemos descobrir um percentil das últimas `N` amostras?

In [ ]:
rng = random.Random(42)

tam = 10000

In [ ]:
%%time


buffer = []
percs75_batch = []

for _ in range(100000):
    v = rng.uniform(-100, 100)
    buffer.append(v)
    if len(buffer) <= tam:
        continue
        
    percs75_batch.append(np.quantile(buffer, q=0.75))
    buffer.pop(0)

Podemos fazer um pouco melhor que isso.

In [ ]:
rng = random.Random(42)

In [ ]:
%%time

qtl = stats.RollingQuantile(window_size=tam, q=0.75)
percs75_incr = []

for _ in range(100000):
    v = rng.uniform(-100, 100)
    qtl.update(v)
    if len(qtl) <= tam:
        continue
        
    percs75_incr.append(qtl.get())

In [ ]:
s_erros = 0

for batch, incr in zip(percs75_batch, percs75_incr):
    s_erros += (batch - incr)

s_erros

Espero tê-los convencido! O [módulo](https://riverml.xyz/dev/api/overview/#stats) `stats` do River tem várias funções úteis para sumarização de dados de forma incremental. Vale a pena checar! 🧐

A razão pela qual eu quis mostrar tudo isso é porque vários desses algoritmos são os "tijolos" que compõem os algoritmos de aprendizado de máquina. Uma receita breve para online learning:

- A junção de métricas estatísticas incrementais
- Algumas grandezas probabilísticas
- Algumas teorias de aprendizado de máquina que são genéricas e não limitadas ao cenário in-batch
- Gradiente descendente
- *otras cositas*

Com isso tudo é possível formar boa parte dos algoritmos de aprendizado incremental. Mas isso é extremamente genérico de se dizer, não é? Bom, no fim das contas, o aprendizado de máquina tradicional também "se resume" a algumas coisas pontuais, mas que são extremamente abrangentes por si só.

Que tal entendermos um pouco de como os exemplos anteriores funcionam e por que eles funcionam?

## 3.2. Uma (extremamente) breve reflexão sobre complexidade dos algoritmos

Quanto custa para calcular a variância (no caso, desvio padrão) e o percentil nos exemplos anteriores?

**Modo batch**

Variância:  $\dfrac{\sum_i^N (x_i - \bar{x})}{N - 1}$

- Média é calculada com o custo $O(n)$ -> todos os elementos devem ser somados e divididos pelo total de observações
- Com a média em mãos, precisamos calcular o desvio de cada elemento para a média e somar tais desvios: $O(n)$
- No fim das contas, o custo final é $O(n)$ (na notação assintótica), mas tivemos que percorrer todos os dados duas vezes (e, consequentemente, armazená-los).
- Esse algoritmo também apresenta problemas de estabilidade, quando o número de observações é muito grande. Pode gerar erros numéricos de aproximação.

Percentil:

- Pegue um exemplo novo e remova o mais antigo
- Ordene os dados: algo em torno de $O(n\log n)$ (n é o tamanho do buffer)
- Encontre a posição correta, interpole e retorne

Se esse processo é repetido várias e várias vezes...

**Modo incremental**


Variância (algoritmo de Welford):

- Precisamos de algumas variáveis:
    - $n$: número de observações
    - $\overline{x}_n$: a média da amostral, após $n$ observações
    - $M_{2, n}$: estatística de segunda ordem que gerará a variância
- As atualizações das estatísticas se dão na seguinte forma:
    - $\overline{x}_n = \overline{x}_{n-1} + \dfrac{x_n - \overline{x}_{n-1}}{n}$
    - $M_{2,n} = M_{2,n-1} + (x_n - \overline{x}_{n-1})(x_n - \overline{x}_n)$
- E as estatísticas são inicializadas da seguinte forma:
    - $\overline{x}_{0} \leftarrow 0$
    - $M_{2,0} \leftarrow 0$
- Para obter a variância amostral basta usar: $s_n^2 = \dfrac{M_{2,n}}{n-1}$, para todo $n > 1$
- De brinde, obtemos um preditor robusto para a média 🤓


Percentil:

- Mantemos dois buffers
   - Um com os dados na ordem em que chegam
   - Outro com os dados ordenados: o custo de inserção de um novo ponto em um vetor já ordenado é $O(\log n)$
   - A remoção de um valor no buffer ordenado é $O(n)$. No buffer não-ordenado é $O(1)$
   - O cálculo do percentil usa o buffer ordenado
   
No fim das contas, sempre buscamos que cada atualização em uma métrica ou modelo de aprendizado tenha um custo constante ($O(1)$). Se isso não for possível, então um custo sub-linear é o objetivo!

---

Antes de prosseguirmos, vou mostrar mais uma aplicação legal de métricas incrementais, com requintes de processamento distribuido.

**Situação hipotética:**

E se estivéssemos calculando estatísticas de alguma coisa, mas a coleta era feita de forma separada? Por exemplo, estamos calculando a variância de alguma coisa a nível estadual, mas a coleta é feita por município?

(Eu sou paranaense)

In [ ]:
rng = random.Random(7)

In [ ]:
dados_ca = [rng.gauss(1500, 200) for _ in range(15000)]
dados_lndrn = [rng.gauss(2500, 500) for _ in range(600000)]

np.var(dados_ca), np.var(dados_lndrn)

E se quisermos calcular a variância (ou média) total de todos os municípios?

In [ ]:
dados_parana = []

# dados_parana.extend(dados_abatiá)
# ...
dados_parana.extend(dados_ca)
# ...
dados_parana.extend(dados_lndrn)
# ...
# dados_parana.extend(dados_xambrê)

len(dados_parana)

In [ ]:
np.var(dados_parana)

In [ ]:
np.mean(dados_parana)

Nós já sabemos como fazer esse mesmo processo de forma incremental:

In [ ]:
# estou considerando que todos os dados foram coletados, mas poderia usar o padrão, ddof=1
var_ca = stats.Var(ddof=0)
var_lndrn = stats.Var(ddof=0)

for p in dados_ca:
    var_ca.update(p)
    
for p in dados_lndrn:
    var_lndrn.update(p)

In [ ]:
var_ca.get(), var_lndrn.get()

Posso usar `stats.Mean` para obter a média. Mas se lembrarmos bem, o algoritmo de Welford tem um preditor de média "lá no meio", não tem?

Voilà!

In [ ]:
var_ca.mean.get(), np.mean(dados_ca)

Agora vem a parte mais legal 🙃

Como obtemos as estatísticas para o estado inteiro?

In [ ]:
# Imagine que temos um for somando as estatísticas de todos os estados

var_parana = var_ca + var_lndrn  # + os outros municipios

var_parana.get(), np.var(dados_parana)

In [ ]:
var_parana.mean.get(), np.mean(dados_parana)

E se a bela e formosa cidade de Cândido de Abreu tivesse sido esquecida? Os dados tivessem sido perdidos, sei lá...

(quando eu era criança, minha cidade nem aparecia nos mapas impressos do Paraná, vai saber...)

In [ ]:
var_ca_backup = var_parana - var_lndrn  # - os outros municipios

In [ ]:
var_ca.get(), var_ca_backup.get()

In [ ]:
var_ca.mean.get(), var_ca_backup.mean.get()

No fim das contas, sempre existe um trade-off entre memória e tempo de processamento (e a quantidade de "passadas" nos dados) e a precisão dos resultados obtidos. Muitos dos algorítmos de online learning incluem um parâmetro ($\delta$) que indica a porcentagem de "certeza" que você deseja ter nos seus resultados. Em geral, quanto mais certeza, mais tempo leva para tomar as decisões.


# 4. Por que usar dicionários (ou, por que usar uma representação esparsa)?

Um as primeiras coisas que podem chamar a atenção no River é o tipo de dados que é utilizado: dicionários python.

- Chave x valor: chaves são únicas
- Valores acessados via chave, ao invés de índice
- Dicionários são estruturas naturalmente esparsas
- Diferentemente de arrays, vetores, matrizes (e etc.) não possuem uma ordenação explícita
- Podem ser extendidos
- Podem ter variáveis de tipo misto

Exemplos:

In [ ]:
from datetime import datetime

x = {
    "batata": 3,
    "carro": 2,
    "data": datetime.now(),
    "sim_ou_não": "sim"
}

x

In [ ]:
x["mais um"] = True
x

In [ ]:
del x["data"]
x

Esse tipo de estrutura é muito parecida com o popular formato JSON (para transferência de dados)!

Que tal compararmos com o que é mais usual?

In [ ]:
data = load_wine()

X, y = data.data, data.target

X[0, :], data.feature_names

In [ ]:
y[0], data.target_names

Primeiramente, vamos colocar o sklearn à prova.

In [ ]:
X_tr, y_tr = X[:-2, :], y[:-2]
X_ts, y_ts = X[-2:, :], y[-2:]

X_tr.shape, X_ts.shape

In [ ]:
from sklearn.naive_bayes import GaussianNB

nb = GaussianNB()

nb.fit(X_tr, y_tr)

nb.predict(X_ts)

Mas, e se uma feature estivesse faltando?

In [ ]:
try:
    nb.predict(X_ts[:, 1:])
except ValueError as error:
    print(error)

Em Online Learning nós não deveríamos ficar nos preocupando com "coisa pouca" assim. Brincadeiras a parte, em um cenário de aprendizado online, sensores podem falhar, novos dados podem surgir, entre muitas outras coisas que podem alterar os dados de entrada. Precisamos de uma representação e modelos robustos quanto a isso!

Com o River, a vasta maioria dos modelos está pronta para lidar com esse tipo de problema! 🎉

In [ ]:
from river import naive_bayes

gnb = naive_bayes.GaussianNB()
dataset = stream.iter_sklearn_dataset(load_wine())

rng = random.Random(42)

# Chance de feature ser ignorada
del_chance = 0.2

n_linhas_incompletas = 0
for i, (x, y) in enumerate(dataset):
    if i == 176:
        break
    
    x_copy = x.copy()
    aux = 0
    for xi in x:
        if rng.random() <= del_chance:
            del x_copy[xi]
            aux = 1
        
        # Atualiza o número de linhas incompletas
        n_linhas_incompletas += aux
    
    gnb.learn_one(x_copy, y)

In [ ]:
x, y

In [ ]:
gnb.predict_proba_one(x)

Vamos bagunçar a última amostra:

In [ ]:
x, y = next(dataset)
list(x.keys())

Vou tirar um cópia do `x` e remover umas coisas

In [ ]:
x_copy = x.copy()

del x_copy["malic_acid"]
del x_copy["hue"]
del x_copy["flavanoids"]

x_copy

Será que o nosso modelo dá conta do recado?

In [ ]:
gnb.predict_proba_one(x_copy), y

E se ao invés de diminuir o número de features aumentasse?

In [ ]:
x["primeira extra"] = 7.89
x["segunda extra"] = 2

x

In [ ]:
gnb.learn_one(x, y)

gnb.predict_one({"primeira extra": 7.8, "segunda extra": 1.5})

In [ ]:
np.unique(data.target, return_counts=True)

Cada tipo de modelo implementa estratégias diferentes para lidar com atributos emergentes ou faltantes!

No nosso exemplo, "1" era a classe majoritária. Essa foi a escolha feita pelo GaussianNB, já que há pouca informação acerca dos dois atributos extra que adicionamos. Mas eles já fazem parte do modelo e serão atualizados caso novas observações cheguem!

In [ ]:
gnb.gaussians

# 5. Como avaliar um modelo?

Já vimos em exemplos passados que os dados foram primeiramente utilizados para teste e depois para treino. Nada de Cross-validation, Leave-one-out, holdout e coisas do tipo.

Esse tipo de avaliação se aproxima mais de um cenário de produção real. Normalmente, obtemos um dado sem a sua label (no caso supervisionado, obviamente) e precisamos fazer predições. Só depois de um tempo a verdadeira label daquele exemplo será conhecida.

Nos exemplos até aqui, nós assumimos que logo após o treinamento do modelo com uma amostra, a label é revelada. No entanto, um cenário ainda mais realístico é aquele onde existe um *delay* entre a predição e a chegada da label. Em alguns casos, certos exemplos podem nunca ter sua label obtida!

Esse tipo de estratégia de avaliação é chamada de "validação progressiva" (*progressive validation*) ou "*prequential*".

Deixo esse [ótimo post do Max Halford](https://maxhalford.github.io/blog/online-learning-evaluation/), onde esse tema é discutido em detalhes.

No River, podemos utilizar a função `progressive_val_score` disponível no módulo `evaluate` para realizar validações progressivas! Daqui para frente, essa função será utilizada.

Obviamente, podemos também escrever alguns `for` e customizar a estratégia de avaliação. Fica de acordo com o critério e a necessidade de cada um!

In [ ]:
from river import evaluate
from river import metrics
from river import synth


def label_delay(x, y):
    return rng.randint(0, 100)


rng = random.Random(8)
dataset = synth.RandomRBF(seed_sample=7, seed_model=9)
model = tree.HoeffdingTreeClassifier()

# Podemos combinar métricas com operadores de pipeline!
metric = metrics.Accuracy() + metrics.MicroF1() + metrics.BalancedAccuracy()

evaluate.progressive_val_score(
    dataset=dataset.take(50000),
    model=model,
    metric=metric,
    print_every=5000,
    show_memory=True,
    show_time=True,
    delay=label_delay
)

In [ ]:
model.draw()

# 6. Concept drift

Um dos aspectos centrais em online learning reside no fato que esperamos que as distribuições dos dados podem ser não-estacionárias. Mas o que isso significa?

Primeiro, vamos pensar num exemplo de distribução estacionária:

> Sabe quando ouvimos falar que a GRANDE-EMPRESA-HYPADA lançou uma rede neural MASTER-BLASTER-ULTRA-POWER-3, com 3 troçalhões de parâmetros, treinada por 3 meses com a energia suficiente para alimentar cidades? E sabe quando ouvimos falar que a base de dados utilizada foi gigante?

Pois bem, os dados não mudam. As regras linguísticas no textos de entrada (supondo um modelo de linguagem natural) ou a semântica visual das imagens (supondo um modelo de visão computacional), ou qualquer outro exemplo, tudo isso é estático. Um cachorro continuará sendo um cachorro (independente do ângulo -- dependendo da forma de captação das imagens isso poderia ser um problema). Uma palavra tem um conjunto de sinônimos e significados.

A regra do jogo não-muda. Mas e se mudasse?

Essa mudança, ou deriva de conceito (concept drift) pode ocorrer em problemas do mundo real. Exemplo:

- padrões de consumo (papel higiênico e alcool em gel no início da pandemia)
- questões relacionadas a clima: energia sustentável
- trânsito, rotas

Umas das áreas de pesquisa (sim, uma área completa de pesquisa) em online learning busca criar detectores para essas situações, bem como algoritmos de aprendizado de máquina que sejam capazes de se adaptar mudanças de conceitos.

Estou longe de ser um especialista nesse tópico, mas vou dar um panorama de como as coisas acontecem.

Suponha que temos um problema de classificação e estamos monitorando os erros do nosso classificador. Nós assinalamos com $0$ quando o modelo acerta (erro zero) e $1$ quando o modelo erra.

In [ ]:
rng = random.Random(8)

for _ in range(10):
    print(rng.choices([0, 1], weights=[0.7, 0.3])[0])

Podemos passar esses valores para um detector de drifts:

In [ ]:
from river import drift

detector = drift.ADWIN(delta=0.01)

vals = rng.choices([0, 1], weights=[0.7, 0.3], k=500)
for i, v in enumerate(vals):
    in_drift, _ = detector.update(v)
    
    if in_drift:
        print(f"Drift detectado em {i}")

E se a distribuição mudar?

In [ ]:
detector = drift.ADWIN(delta=0.05)

vals = rng.choices([0, 1], weights=[0.7, 0.3], k=500)
vals.extend(rng.choices([0, 1], weights=[0.2, 0.8], k=500))
for i, v in enumerate(vals):
    in_drift, _ = detector.update(v)
    
    if in_drift:
        print(f"Drift detectado em {i}")
        # Pratica usual
        detector.reset()

O ADWIN é um dos detectores mais utilizados, mas existem várias alternativas. Além disso, normalmente, os detectores são utilizados como componentes na criação de modelos preditivos. A forma na qual eles são utilizados difere de modelo para modelo. Uma característica é comum entre a maioria dos detectores de drift: eles são univariados.

# 7. Exemplos de algoritmos

Existem muitos algoritmos de aprendizado online. Uns são bem diferentes, outros são modificações de outros algoritmos. Assim como em aprendizado de máquina tradicional, existem opções para todo gosto.

As listas a seguir não são extensivas de modo algum. São apenas o que eu fui lembrando, tenho certa familiaridade, e o que eu vejo de popular por aí (obviamente com o meu viés pessoal). Uma boa pedida é consultar a documentação do River para ver o que há de disponível por lá!

Mostrarei de forma breve seu uso e, quando aplicável, quais dos seus hiper-parâmetros são os mais "sensíveis", na minha experiência.

---
---

**Dica:** Se **dúvidas** específicas surgirem acerca de alguma família, me **interrompam**. Posso **comentar** mais **detalhes** e ir deixando a conversa mais técnica.

---
---

## 7.1. Classificação

Abordaremos uma lista de algoritmos para classificação. Se um algoritmo foi projetado apenas para classificação binária e o problema em questão é multiclasses, as ferramentas do módulo `multiclass` podem ser utilizadas como um wrapper:

- `OneVsOneClassifier`
- `OneVsRestClassifier`
- `OutputCodeClassifier`

### 7.1.1. Logistic Regression

Versão incremental de Logistic Regression (para classificação binária). Pode ser treinada com vários otimizadores. Por padrão usa o `Stochastic Gradient Descent`.


Não tenho muitas dicas específicas acerca dessa família. Mas todo o conhecimento ligado a redes neurais tende a ser útil aqui.

É possível ajustar:

- O otimizador
- O algortmo de inicialização
- Adicionar regularização L2
- A função de perda
- entre outros

**Exemplo:**

In [ ]:
from river import datasets
from river import linear_model
from river import optim
from river import preprocessing

dataset = datasets.Phishing()

model = (
    preprocessing.StandardScaler() |
    linear_model.LogisticRegression(optimizer=optim.Adam(), l2=0.01)
)

metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=100)

Podemos inspecionar como as decisões foram feitas:

Uma variante de Logistic Regression é SoftMax Regression (voltada para problemas multi-classe e mais robusta a outliers):

In [ ]:
dataset = datasets.ImageSegments()

model = (
    preprocessing.StandardScaler() |
    linear_model.SoftmaxRegression()
)

metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=200, show_memory=True, show_time=True)

### 7.1.2. Hoeffding Tree (e variantes)

Estão entre os modelos mais populares em stream mining. Historicamente um dos algoritmos mais usados, senão o mais usado. Minha pesquisa de doutorado contempla árvores de decisão e ensembles online. Existem vários classificadores no River baseados em árvores de decisão. Grande parte deles pertencem a família das *Hoeffding Trees*.

Hoeffding Trees (HT) têm esse nome porque elas se baseiam numa grandeza estatística chamada *Hoeffding bound* para decidir quando fazer uma divisão. Essa medida traz indícios que um split feito de forma incremental seria o mesmo que seria feito em um algoritmo de árvore tradicional (desde que observações suficientes sejam feitas).

Existem três variantes principais das HTs:

- Hoeffding Tree: versão padrão
- Hoeffding Adaptive Tree: adiciona detectores de deriva de conceito em cada nó de decisão. Se uma mudança é detectada, uma nova sub-árvore é treinada (no *background*) e eventualmente virá a substituir o ramo afetado.
- Extremely Fast Decision Tree: Faz splits mais rapidamente, mas periodicamente revisita as decisões feitas e vai reconstruindo as árvores.

**Principais parâmetros para ajuste comum a todas as HTs:**

- `grace_period`: intervalos entre tentativas de *splits*.
- `split_confidence`: o quão "certas" sobre uma decisão as árvores devem estar, de forma fazer um *split*. A "certeza" é dada por `1 - split_confidence`.
- `tie_threshold`: threshold abaixo do qual um split será feito, mesmo se não houver plena certeza que o melhor candidato para *split* é realmente a melhor escolha.
- `max_depth`: profundeza máxima que uma árvore pode alcançar.

Eu escrevi um [tutorial](https://riverml.xyz/dev/user-guide/on-hoeffding-trees/) sobre HTs que aprofunda mais o que é possível se fazer com esse tipo de preditor.

**Exemplo:**

In [ ]:
from river import tree


dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)
model = tree.HoeffdingAdaptiveTreeClassifier(seed=42)
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

Como já vimos anteriormente, podemos visualizar a estrutura da árvore.

In [ ]:
model.draw()

Também é possível inspecionar como uma decisão é tomada:

In [ ]:
dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)

x, y = next(dataset)

print(model.debug_one(x))

### 7.1.3. Naive Bayes

O Naive Bayes é inerentemente incremental. Assim, é possível atualizar as contagens ou estimativas das distribuições de forma incremental.

Modelos disponíveis:

- BernoulliNB, ComplementNB e MultinomialNB: voltados para os casos onde os dados são contagens (bag-of-words), ou a saída do TF-IDF.
-  GaussianNB: discretiza as features numéricas utilizando uma distribuição gaussiana por classe.

**Exemplo:**

In [ ]:
dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)


# Note que o modelo selecionado não tem trata a deriva de conceitos
model = naive_bayes.GaussianNB()
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

### 6.1.4. Adaptive Random Forest

A adaptive random forest (ARF) é uma versão incremental do algoritmo Random Forest e combinas os seguintes ingredientes:

- Hoeffding Trees com pitadas de aleatoriadade, como *base learners* (ingrediente principal)
- Detectores de deriva de conceito por árvore
    - árvores "reserva" são treinadas em *background* caso drifts são detectados
- Bagging na versão online

Além de todos os parâmetros das HTs, alguns parâmetros notáveis da ARF são:

- `warning_detector` e `drift_detector`: detectores de warning e drift
- `n_models`: o número de árvores
- `max_features`: o número de features que será considerado por árvore durante *splits*

**Exemplo:**

In [ ]:
from river import ensemble


dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)

model = ensemble.AdaptiveRandomForestClassifier(seed=8)
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

### 7.1.5. Streaming Random Patches

Na ARF e random forest em geral, cada vez que um nó da árvore é dividido, um novo sub-conjunto das features é selecionado. Apenas esse subconjunto é considerado nas decisões. Esse processo pode ser referido como amostragem local de features. A (A)RF também faz amostragem de instâncias com reposição (*bagging*).

O Streaming Random Patches (SRP) também faz amostragem de instâncias via bagging. No entanto, a amostragem de features é global: feita apenas uma vez por modelo no ensemble. Além disso, o SRP não limita os membros do ensemble à árvores de decisão: qualquer classificador pode ser utilizado 😁

A detecção e reação a derivas de conceito (concept drift) é igual ao mecanismo adotado na ARF.

Quanto aos hiperparâmetros mais notáveis:

- `model`: o classificador base
- `n_models`: o número de classificadores no ensemble
- `warning_detector` e `drift_detector`: o mesmo que na ARF

**Exemplo:**

In [ ]:
from river import neighbors

dataset = synth.RandomRBFDrift(
    seed_model=7, seed_sample=8, change_speed=0.0001, n_classes=3,
).take(15000)


model = ensemble.SRPClassifier(
    neighbors.KNNClassifier(),
    n_models=5, seed=8,
    training_method="subspaces"
)
metric = metrics.Accuracy()

evaluate.progressive_val_score(dataset, model, metric, print_every=1000, show_memory=True, show_time=True)

## 7.2. Regressão


Para regressão, vou adotar o mesmo dataset em todos os casos, para termos uma ideia das capacidades.

In [ ]:
def get_friedman():
    return synth.Friedman(seed=101).take(20000)

### 7.2.1. Linear Regression


A regressão linear permite a exploração de vários parâmetros:

- otimizadores
- funções de perda
- regularização
- estratégias para atualização do fator de aprendizado
- entre outras coisas

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | linear_model.LinearRegression()

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | linear_model.LinearRegression(
    intercept_lr=0.02, intercept_init=3.5, l2=0.02
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

### 7.2.2. Hoeffding Tree

(minha pesquisa se situa aqui)

Temos também versões da HT para regressão. Os principais modelos para regressão de único alvo são:

- `HoeffdingTreeRegressor`: versão padrão do regressor.
- `HoeffdingAdaptiveTreeRegressor`: extende o anterior de forma similar à versão de classificação.

Além dos parâmetros já apresentados na árvore de classificação, outros parâmetros importantes são:

- `leaf_prediction`: seleciona a estratégia de predição -- regression ou model tree
- `leaf_model`: modelo preditivo utilizado nas folhas, no caso das model trees
- `splitter`: algoritmo utilizado para lidar com *splits* em atributos numéricos

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | tree.HoeffdingTreeRegressor()

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

Como usual, podemos inspecionar os modelos baseados em árvores de decisão:

In [ ]:
x, y = next(get_friedman())

print(model.debug_one(x))

In [ ]:
model["HoeffdingTreeRegressor"].draw()

Vou mudar a estratégia de predição, apenas por ilustração.

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | tree.HoeffdingTreeRegressor(
    leaf_model=linear_model.PARegressor(C=0.1, eps=0.0001)
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

Querem conhecer um pouco da minha pesquisa?

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | tree.HoeffdingTreeRegressor(
    leaf_prediction="adaptive",
    splitter=tree.splitter.QOSplitter(2, allow_multiway_splits=True)  # <- parte da minha pesquisa está focada aqui
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

**Minha pesquisa:**

> ### *Diminuir os custos computacionais de regressores baseados em árvores e regras de decisão, bem como ensembles destes preditores.*

### 7.2.3. AMRules

Adaptive Model Rules.

(minha pesquisa se situa aqui também)

Cria regras de decisão baseadas na Hoeffding bound, assim como as árvores. É capaz de fazer deteção de anomalias! De fato, o AMRule "pula" amostras anomalas durante o treinamento.

Tem alguns ajustes muito parecidos com as árvores de decisão:

- `n_min`: equivalente ao `grace_period`
- `delta`: equivalente ao  `split_confidence`
- `tau`: equivalente `tie_threshold`
- `pred_type`: equivalente `leaf_prediction`
- `pred_model`: equivalente `leaf_model`
- `splitter`

Em adição, outros parâmetros importantes:

- `m_min`: número de instâncias que cada regra deve observar antes de começar a detectar anomalias.
- `drift_detector`: o detector de concept drift utilizado pelas regras de decisão
- `anomaly_threshold`: valor de corte para considerar uma amostra anômala. Instâncias serão consideradas anômalas se o `score` delas for menor que esse limiar.
- `ordered_rule_set`: define se apenas a primeira regra que dá "*match*" em uma instância será utilizada, ou todas as regras.

In [ ]:
from river import rules

metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | rules.AMRules(
    splitter=tree.splitter.TEBSTSplitter(digits=1),  #  <- olha minha pesquisa aqui também
    drift_detector=drift.ADWIN(),
    ordered_rule_set=False,
    m_min=100,
    delta=0.01
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

Também podemos inspecionar o modelo:

In [ ]:
x, y = next(get_friedman())

print(model.debug_one(x))
print(f"Valor esperado: {y}")

In [ ]:
x_scaled = model["StandardScaler"].transform_one(x)

model["AMRules"].anomaly_score(x_scaled)

In [ ]:
help(rules.AMRules.anomaly_score)

### 7.2.4. Adaptive Random Forest

(minha pesquisa também está aqui!)

Utiliza uma versão da `HoeffdingTreeRegressor` como preditor base. Tem todos os hiper-parâmetros da árvore de decisão e os parâmetros que já foram apresentados na ARF para classificação.

Além disso, adiciona um parâmetro que tende a impactar bastante os resultados:

- `agregation_method`: como as predições individuais de cada árvore serão combinadas.

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | ensemble.AdaptiveRandomForestRegressor(
    seed=7,
    aggregation_method="median",
    splitter=tree.splitter.QOSplitter(1)
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

### 7.7.5. Streaming Random Patches

(minha pesquisa também está aqui)

É a versão para regressão do SRP apresentado anteriormente. Compartilha os mesmos parâmetros apresentados para o algoritmo de classificação.

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | ensemble.SRPRegressor(
    model=linear_model.LinearRegression(l2=0.02, intercept_init=3.5, intercept_lr=0.04),
    seed=7,
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

Podemos também utilizar Bagging para unir regressores.

In [ ]:
metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | ensemble.BaggingRegressor(
    model=tree.HoeffdingAdaptiveTreeRegressor(splitter=tree.splitter.QOSplitter(0.5), seed=7),
    seed=7,
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

## 7.3. Clustering

Algoritmos incrementais para clustering devem ser capazes de se adaptarem às mudanças constantes nos dados. Por exemplo, alguns clusters podem surgir, outros desaparecerem. É uma área ativa de pesquisa, com muitos algoritmos disponíveis!

Apresentarei um algoritmo particional baseado em protótipos, para ilustrar.

### 7.3.1. k-Means

Existem várias versões incrementais do k-Means. A versão no River adiciona um parâmetro chamado `halflife` que controla o nível das atualizações incrementais.

In [ ]:
from river import cluster

metric = metrics.cluster.Silhouette()
model = cluster.KMeans(seed=7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric)

Vamos diminuir o número de clusters.

In [ ]:
metric = metrics.cluster.Silhouette()
model = cluster.KMeans(n_clusters=3, seed=7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric)

E aumentar o valor de `halflife`.

In [ ]:
metric = metrics.cluster.Silhouette()
model = cluster.KMeans(n_clusters=3, seed=7, halflife=0.7)


for x, _ in get_friedman():
    metric.update(x, model.predict_one(x), model.centers)
    model.learn_one(x)

print(metric)

## 7.4. Anomaly detection

Até o momento o River conta com um detector de anomalias, que é a versão incremental da Isolation Forest.


In [ ]:
%%time

from river import anomaly

dataset = datasets.CreditCard()
model = preprocessing.MinMaxScaler() | anomaly.HalfSpaceTrees(seed=42, window_size=20)

limiar_anomalia = 0.8
var = stats.Var()

fraudes = []
fraudes_preditas_static = []
fraudes_preditas_dynamic = []

fraudes_match_static = []
fraudes_match_dynamic = []

metric_static = metrics.BalancedAccuracy() + metrics.Precision() + metrics.Recall() + metrics.F1()
metric_dynamic = metrics.BalancedAccuracy() + metrics.Precision() + metrics.Recall() + metrics.F1()

for i, (x, y) in enumerate(dataset):
    score = model.score_one(x)
    model.learn_one(x)
    
    pred_static = 1 if score > limiar_anomalia else 0
    
    std = var.get() ** 0.5
    pred_dynamic = 1 if score > var.mean.get() + 1.5 * std else 0
    
    if y == 1:
        fraudes.append(i)
        
        if pred_static == 1:
            fraudes_match_static.append(i)
        
        if pred_dynamic == 1:
            fraudes_match_dynamic.append(i)
                
    
    if pred_static == 1:
        fraudes_preditas_static.append(i)
        
    if pred_dynamic == 1:
        fraudes_preditas_dynamic.append(i)
    
    metric_static.update(y, pred_static)
    metric_dynamic.update(y, pred_dynamic)
    var.update(score)
    
    if i >= 100000:
        break

In [ ]:
print(f"Número de amostras: {i}")
print(f"Número de anomalias: {len(fraudes)}")
print(f"Número de anomalias detectadas (estático): {len(fraudes_preditas_static)}")
print(f"Número de anomalias detectadas (dinâmico): {len(fraudes_preditas_dynamic)}")
print(f"Match detecções (estático): {len(fraudes_match_static) / len(fraudes)}")
print(f"Match detecções (dinâmico): {len(fraudes_match_dynamic) / len(fraudes)}")

In [ ]:
metric_static

In [ ]:
metric_dynamic

Vamos checar como estava o limiar dinâmico no final?

In [ ]:
var.mean.get() + 1.5 * (var.get() ** 0.5)

### Leitura extra!

Um membro do [Spike](https://www.spikelab.xyz/) escreveu um tutorial muito bom sobre detecção de anomalias usando River: https://medium.com/spikelab/anomalies-detection-using-river-398544d3536


Ele aborda um cenário de produção real e vai muito além do meu simples exemplo. Recomendo muito a leitura!

# 8. Expert stuff

O módulo `expert` traz utilidades para seleção e combinação de modelos.

Abordarei dois exemplos:

1. Seleção de modelos
2. Combinação de modelos


## 8.1. Selecionando um modelo de forma online

Suponha que eu tenho um conjunto de hiper-parâmetros que gostaria de avaliar, mas não gostaria de rodar todos os modelos para descobrir qual é o melhor. Vamos usar a regressão linear como exemplo aqui.

In [ ]:
model = (
    preprocessing.StandardScaler() |
    linear_model.LinearRegression(intercept_lr=.01, intercept_init=3.5)
)

In [ ]:
from river import utils

models = utils.expand_param_grid(model, {
    'LinearRegression': {
        'optimizer': [
            (optim.SGD, {'lr': [.1, .01, .005]}),
            (optim.Adam, {'beta_1': [.01, .001], 'lr': [.1, .01, .001]}),
            (optim.NesterovMomentum, {'rho': [.8, .9, .95], 'lr': [.001, .02, .04, .1]}),
        ],
        'l2': [.0, .1, .01, .001],
        'loss': [optim.losses.Absolute(), optim.losses.Squared()]
    }
})

In [ ]:
len(models)

In [ ]:
from river import expert


metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = expert.SuccessiveHalvingRegressor(
    models=models,
    budget=5000,
    metric=metrics.RMSE()
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

In [ ]:
model.best_model._get_params()

## 8.2. Combinação de modelos

Nós já vimos alguns ensembles, mas também é possível combinar modelos de outras formas.

In [ ]:
regressors = [
    tree.HoeffdingTreeRegressor(
        splitter=tree.splitter.QOSplitter(),
        leaf_prediction="adaptive"
    ),
    rules.AMRules(
        drift_detector=drift.ADWIN(),
        splitter=tree.splitter.QOSplitter()
    ),
    tree.SGTRegressor(
        grace_period=50,
        feature_quantizer=tree.splitter.DynamicQuantizer(),
        init_pred=3.5
    ),
]

metric = metrics.MAE() + metrics.RMSE() + metrics.R2()
model = preprocessing.StandardScaler() | expert.EWARegressor(
    regressors=regressors,
    learning_rate=0.04
)

evaluate.progressive_val_score(
    dataset=get_friedman(),
    model=model,
    metric=metric,
    show_memory=True,
    show_time=True,
    print_every=2000
)

# 9. Pipelines e pré-processamento

Vocês notaram que eu tenho usado muito os operadores "|" e "+" em alguns contextos?

Esses operadores são utilizados para criar pipelines. Eles são algo que chamamos de "sugar syntax".

No River nós podemos usar operadores explícitos, assim como no sklearn.

In [ ]:
from river import compose


model = compose.Pipeline(preprocessing.StandardScaler(), linear_model.LinearRegression())

Vamos supor que no exemplo anterior, algumas das features que seriam passadas ao nosso modelo eram categóricas. Poderíamos aplicar um encoding nesses exemplos para garantir que nosso modelo funcionaria com a regressão linear.

In [ ]:
import numbers

pipeline = compose.Pipeline(
    compose.TransformerUnion(
        compose.SelectType(numbers.Number),
        compose.Pipeline(
            compose.SelectType(str),
            preprocessing.OneHotEncoder()
        ),
    ),
    preprocessing.StandardScaler(),
    linear_model.LinearRegression()
)

pipeline

Que coisa linda né? Eu "escondi o ouro" e propositalmente não mostrei essa visualização anteriormente. Muito útil para *debugging*. Prático? Pode ficar ainda melhor!

Sugar syntax! Vou montar o mesmo pipeline sem usar explicitamente os componentes de `compose`.

In [ ]:
pipeline = (
    (compose.SelectType(numbers.Number) + (compose.SelectType(str) | preprocessing.OneHotEncoder())) |
    preprocessing.StandardScaler() | linear_model.LinearRegression()
)
pipeline

Muito mais legível, na minha humilde opinião!

Existem várias possibilidades com `compose` e Pipelines. Um bom ponto de partida: [The art of using pipelines](https://riverml.xyz/latest/examples/the-art-of-using-pipelines/).

Algumas delas serão melhor abordadas no exemplo da próxima seção. A documentação do módulo é bem completa, vale a pena conferir!

# 10. Exemplo completo

Nesse exemplo iremos aplicar todos os conceitos vistos anteriormente.

**Atenção:**

> O objetivo aqui é aplicação de conhecimento, e não lidar com o problema preditivo em si.

Provavelmente a extração de melhores features melhoraria o desempenho preditivo do regressor.

O dataset a ser utilizado é um subconjunto do [Airlines](https://kt.ijs.si/elena_ikonomovska/data.html). Utilizaremos dois anos do conjunto total. O dataset deve ser extraído para a mesma pasta deste notebook.

O dataset conta com mais de 15 milhões de amostras referentes a vôos realizados no período mencionado. O objetivo é prever o atraso de cada vôo em minutos.

In [ ]:
import calendar
import datetime as dt
import functools
import numbers
import random

from river import compose
from river import drift
from river import evaluate
from river import feature_extraction
from river import linear_model
from river import metrics
from river import preprocessing
from river import rules
from river import stats
from river import stream
from river import time_series
from river import tree

In [ ]:
def drop_some(rng, p):
    def parser(x, rng=rng, p=p):
        if rng.random() <= p:
            return None
        else:
            return float(x)
    return parser


feature_converters =  {
    "Year": int,
    "Month": int,
    "DayofMonth": int,
    "DayofWeek": int,
    "CRSDepTime": str,
    "CRSArrTime": str,
    "UniqueCarrier": str,
    "FlightNum": int,
    "ActualElapsedTime": float,
    "Origin": str,
    "Dest": str,
    "Distance": drop_some(random.Random(7), p=0.01),
    "Diverted": lambda d: "yes" if d == 1 else "no",
    "ArrDelay": float,
} 


dataset = stream.iter_csv(
    "airlines07_08.csv",
    target="ArrDelay",
    converters=feature_converters,
)

In [ ]:
# Funções úteis
def slice_data(dataset, max_samples=2000000):
    dataset = iter(dataset)
    for i, (x, y) in enumerate(dataset):
        yield x, y
        
        if i == max_samples:
            return

def parse_date(x):
    x = x.copy()
    
    # Adiciona digitos faltantes
    departure = x["CRSDepTime"]
    while len(departure) < 4:
        departure = "0" + departure
    
    arrival = x["CRSArrTime"]
    while len(arrival) < 4:
        arrival = "0" + arrival
    
    x.pop("CRSDepTime", None)
    x.pop("CRSArrTime", None)
        
    departure = dt.datetime.strptime(departure, "%H%M")
    arrival = dt.datetime.strptime(arrival, "%H%M")
    duration = dt.timedelta(minutes=x["ActualElapsedTime"])
    
    diff = (arrival - departure) - duration
    x["approx_time_zone_diff_min"] = (diff.seconds // 3600) * 60 + ((diff.seconds // 60) % 60) 
    x["est_delay"] = ((arrival - duration) + diff) - departure
    x["exp_dep_hour"] = departure.hour
    x["exp_dep_minute"] = departure.minute
    x["exp_arr_hour"] = arrival.hour
    x["exp_arr_minute"] = arrival.minute
    x["Month"] = calendar.month_name[x["Month"]]
    x["DayofWeek"] = calendar.day_abbr[x["DayofWeek"] - 1]

    return x

In [ ]:
p_imputer = preprocessing.StatImputer(("Distance", stats.Mean()))
p_time = compose.FuncTransformer(parse_date)
p_discard = compose.Discard("Origin", "Dest", "UniqueCarrier")
p_cat = compose.SelectType(str) | preprocessing.OneHotEncoder()
p_num = compose.SelectType(numbers.Number) | preprocessing.StandardScaler()

p_extraction = (
    feature_extraction.TargetAgg(
        by=["DayofWeek", "UniqueCarrier"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
    + feature_extraction.TargetAgg(
        by=["Origin", "Dest"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
    + feature_extraction.TargetAgg(
        by=["Origin", "UniqueCarrier"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
    + feature_extraction.TargetAgg(
        by=["Dest", "UniqueCarrier"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
    + feature_extraction.TargetAgg(
        by=["Origin", "Dest", "UniqueCarrier"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
    + feature_extraction.TargetAgg(
        by=["Origin", "Dest", "UniqueCarrier", "FlightNum"],
        how=stats.EWMean(0.8),
        target_name="delay"
    )
)

preproc = p_imputer | p_time | (p_extraction + p_discard) | (p_cat + p_num)
preproc

In [ ]:
regressor = rules.AMRules(
    n_min=1000,
    m_min=100,
    drift_detector=drift.ADWIN(0.01),
    pred_type="adaptive",
    splitter=tree.splitter.QOSplitter(0.5),
    tau=0.005,
    ordered_rule_set=False,
    pred_model=linear_model.PARegressor(C=0.7, eps=5)
)

metric = metrics.Rolling(metrics.MAE(), 10000)

model = preproc | regressor
model

In [ ]:
evaluate.progressive_val_score(
    slice_data(dataset),
    model,
    metric,
    show_memory=True,
    show_time=True,
    print_every=10000,
)